Unattended Baggage Claim

Import packages

In [1]:
from matplotlib import pyplot as plt
from imutils.video import FPS
from IPython.display import Video
from IPython.display import HTML
import numpy as np
import argparse
import imutils
import cv2

Display function for Jupyter Notebook

In [2]:
def plt_imshow(title, image):
	# convert the image frame BGR to RGB color space and display it
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	plt.imshow(image)
	plt.title(title)
	plt.grid(False)
	plt.show()

Argument parser

In [3]:
# since we are using Jupyter Notebooks we can replace our argument
# parsing code with *hard coded* arguments and values
args = {
	"input" : "video/unattended-bag.mp4",
	"output" : "output.avi",
	"prototxt": "MobileNetSSD_deploy.prototxt.txt",
	"model": "MobileNetSSD_deploy.caffemodel",
	"confidence" : 0.2
}

In [4]:
# initialize the list of class labels MobileNet SSD was trained to
# detect
CLASSES = ["background",
		   "aeroplane",
		   "bicycle",
		   "bird",
		   "boat",
		   "bottle",
		   "bus",
		   "car",
		   "cat",
		   "chair",
		   "cow",
		   "diningtable",
		   "dog",
		   "horse",
		   "motorbike",
		   "person",
		   "pottedplant",
		   "sheep",
		   "sofa",
		   "train",
		   "tvmonitor"]
# generate a set of bounding box colors
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

# debug message
print("[INFO] loading model...")
# load our serialized model from disk
net = cv2.dnn.readNetFromCaffe(args["prototxt"], args["model"])

[INFO] loading model...


In [5]:
# debug message
print("[INFO] opening video file...")
# grab a reference to the video file
# initialize pointer to output video file
vs = cv2.VideoCapture(args["input"])
writer = None
# and initialize the FPS counter
fps = FPS().start()

[INFO] opening video file...


In [6]:
# loop over the frames from the video stream
while True:
	# grab the next frame
	frame = vs.read()[1]

	# if we did not grab a frame then we have reached the end of the
	# video
	if frame is None:
		break

	# resize the frame to have a maximum width of 300 pixels
	frame = imutils.resize(frame, width=400)

	# grab the frame dimensions and convert it to a blob
	(h, w) = frame.shape[:2]
	# calculate a 300x300 pixel blob from our image
	blob = cv2.dnn.blobFromImage(frame, 0.007843, (300, 300), 127.5)

	# pass the blob through the network and obtain the detections and
	# predictions
	net.setInput(blob)
	detections = net.forward()

 	# loop over the detections
	for i in np.arange(0, detections.shape[2]):
		# first we extract the confidence of the detection
		# there may be multiple detections
		confidence = detections[0, 0, i, 2]

		# if the confidence is above our threshold (ex. 20%),
		if confidence > args["confidence"]:
			# extract the index of the class label from the detection
			idx = int(detections[0, 0, i, 1])
			# then compute the (x, y)-coordinates of
			# the bounding box for the object
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			# extract the XY to draw the rectangle and display the label
			(startX, startY, endX, endY) = box.astype("int")

			# build a text label containing the class name and confidence
			label = "{}: {:.2f}%".format(CLASSES[idx],
				confidence * 100)
			# draw a colored rectangle around the object using the extracted XY
			cv2.rectangle(frame, (startX, startY), (endX, endY),
				COLORS[idx], 2)
			# we want the label to display above the rectangle if there's room
			# else, just below the top
			y = startY - 15 if startY - 15 > 15 else startY + 15
			# overlay the colored text on the image
			cv2.putText(frame, label, (startX, y),
				cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)

	# update the FPS counter
	fps.update()

	# if the video writer is None *AND* we are supposed to write
	# the output video to disk initialize the writer
	if writer is None and args["output"] is not None:
		fourcc = cv2.VideoWriter_fourcc(*"MJPG")
		writer = cv2.VideoWriter(args["output"], fourcc, 20,
			(frame.shape[1], frame.shape[0]), True)

	# if the writer is not None, write the frame with recognized
	# faces to disk
	if writer is not None:
		writer.write(frame)

# stop the timer and display FPS information
fps.stop()
print("[INFO] elapsed time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

# do a bit of cleanup
vs.release()

# check to see if the video writer point needs to be released
if writer is not None:
	writer.release()

[INFO] elapsed time: 11.68
[INFO] approx. FPS: 34.17


In [7]:
!ffmpeg -i output.avi unattended-bag-output.mp4

ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with Apple clang version 12.0.0 (clang-1200.0.32.27)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.3.1_4 --enable-shared --enable-pthreads --enable-version3 --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librtmp --enable-libspeex --enable-libsoxr --enable-videotoolbox --disable-libjack --disable-indev=jack
  libavutil      56. 51.100 /

In [10]:
%%HTML
<video width="320" height="240" controls>
  <source src="output.mp4" type="video/mp4">
</video>